<a href="https://colab.research.google.com/github/WhiteAndBlackFox/LearningPyTorch/blob/DDBDO/Dataset%2C_Dataloader%2C_BatchNorm%2C_Dropout%2C_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset, Dataloader, BatchNorm, Dropout, Оптимизация

## Используя датасет недвижимости (sklearn.datasets.fetch_california_housing)
Необходимо:
* Создать Dataset для загрузки данных
* Обернуть его в Dataloader
* Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
* Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

При этом train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25.

### Подключаем необходимые библиотеки

In [59]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.optim import Adam, RMSprop, SGD
from torch.utils.data import DataLoader, Dataset

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Глобальные переменные

In [60]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHES = 10
LR = 0.01

### Создадим Dataset

In [61]:
class CHDataset(Dataset):
  def __init__(self, *init_datasets):
    assert all(init_datasets[0].size(0) == init_dataset.size(0) for init_dataset in init_datasets), "Несоотвутствует размерность среди dataset"
    self._base_datasets = init_datasets
  
  def __len__(self):
    return self._base_datasets[0].size(0)
  
  def __getitem__(self, idx):
      return tuple(base_dataset[idx] for base_dataset in self._base_datasets)

### Опишем нейронную сеть

In [62]:
class CHNet(nn.Module):
  def __init__(self) -> None:
      super(CHNet, self).__init__()
      self.block_1 = nn.Sequential(
          nn.Linear(in_features=8, out_features=100, bias=True),
          nn.Dropout(0.1),
          nn.BatchNorm1d(100),
          nn.ReLU())
      self.block_2 = nn.Sequential(
          nn.Linear(in_features=100, out_features=100, bias=True),
          nn.Dropout(0.2),
          nn.BatchNorm1d(100),
          nn.ReLU())
      self.block_3 = nn.Sequential(
          nn.Linear(in_features=100, out_features=60, bias=True),
          nn.Dropout(0.2),
          nn.BatchNorm1d(60),
          nn.ReLU())
      self.block_4 = nn.Sequential(
          nn.Linear(in_features=60, out_features=30),
          nn.Dropout(0.2),
          nn.BatchNorm1d(30),
          nn.ReLU())
      self.predict = nn.Sequential(
          nn.Linear(in_features=30, out_features=1, bias=True),
          nn.BatchNorm1d(1),
          nn.ReLU())
  
  def forward(self, inp):
    out = self.block_1(inp)
    out = self.block_2(out)
    out = self.block_3(out)
    out = self.block_4(out)
    out = self.predict(out)
    return out[:, 0]

### Функция для оценки оптимизации

In [63]:
def train_loop(train_loader, test_loader, net, optimizer):
  loss_fn = nn.MSELoss()
  best_acc = {'train': None, 'test': None}
  net.train()
  for epoch in range(EPOCHES):
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)

        # обнуляем градиент
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        
        # выводим статистику о процессе обучения
        if i % 150 == 0 or (i + 1) == len(train_loader):    # печатаем каждые 150 mini-batches
            net.eval()

            test_loss, test_running_total, test_loss  = 0.0, 0.0, 0.0
            for y, (out_test, lbl_test) in enumerate(test_loader):
                test_outputs = net(out_test)
                test_loss += loss_fn(test_outputs, lbl_test)
                test_running_total += len(lbl_test)
            
            res_loss_train = running_loss / running_items
            res_loss_test = test_loss / test_running_total
            
            if best_acc['train'] is None or res_loss_train < best_acc['train']:
              best_acc['train'] = res_loss_train
            
            if best_acc['test'] is None or res_loss_test < best_acc['test']:
              best_acc['test'] = res_loss_train

            print(f'Epoch [{epoch + 1}/{EPOCHES}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {res_loss_train:.3f}. '\
                  f'Test acc: {res_loss_test:.3f}.')
            
            running_loss, running_items = 0.0, 0.0
            net.train()
  print(f"Best acc train: {best_acc['train']:.3f}. Best acc test: {best_acc['test']:.3f}")
  print('Training is finished!')

### Начало анализа

In [64]:
california_housing = fetch_california_housing()
# Разделим на тестовые и тренеровочные данные
X_train, X_test, y_train, y_test = train_test_split(california_housing.data, california_housing.target, test_size=0.25, random_state=13)

In [65]:
# Нормализуем данные и подготовим их для дальнейшего использования в нашем dstaset
scale = StandardScaler()
X_train_s = scale.fit_transform(X_train)
X_test_s = scale.transform(X_test)

In [66]:
train_xt = torch.from_numpy(X_train_s.astype(np.float32)).to(DEVICE)
train_yt = torch.from_numpy(y_train.astype(np.float32)).to(DEVICE)

test_xt = torch.from_numpy(X_test_s.astype(np.float32)).to(DEVICE)
test_yt = torch.from_numpy(y_test.astype(np.float32)).to(DEVICE)

In [67]:
train_dataset = CHDataset(train_xt, train_yt)
test_dataset = CHDataset(test_xt, test_yt)

In [68]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=2, drop_last=True)

### Дальшей начнем анализ по таким оптимизаторам как Adam, RMSProp, SGD и SGD + Momentum

### Adam

In [69]:
net = CHNet().to(DEVICE)
optimizer = Adam(net.parameters(), lr=LR)

In [70]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/10]. Step [1/241]. Loss: 0.074. Test acc: 0.077.
Epoch [1/10]. Step [151/241]. Loss: 0.025. Test acc: 0.009.
Epoch [1/10]. Step [241/241]. Loss: 0.010. Test acc: 0.009.
Epoch [2/10]. Step [1/241]. Loss: 0.007. Test acc: 0.009.
Epoch [2/10]. Step [151/241]. Loss: 0.008. Test acc: 0.010.
Epoch [2/10]. Step [241/241]. Loss: 0.008. Test acc: 0.007.
Epoch [3/10]. Step [1/241]. Loss: 0.008. Test acc: 0.007.
Epoch [3/10]. Step [151/241]. Loss: 0.008. Test acc: 0.007.
Epoch [3/10]. Step [241/241]. Loss: 0.007. Test acc: 0.006.
Epoch [4/10]. Step [1/241]. Loss: 0.008. Test acc: 0.006.
Epoch [4/10]. Step [151/241]. Loss: 0.007. Test acc: 0.010.
Epoch [4/10]. Step [241/241]. Loss: 0.007. Test acc: 0.010.
Epoch [5/10]. Step [1/241]. Loss: 0.005. Test acc: 0.010.
Epoch [5/10]. Step [151/241]. Loss: 0.007. Test acc: 0.011.
Epoch [5/10]. Step [241/241]. Loss: 0.007. Test acc: 0.014.
Epoch [6/10]. Step [1/241]. Loss: 0.007. Test acc: 0.014.
Epoch [6/10]. Step [151/241]. Loss: 0.007. Test acc:

### RMSProp

In [71]:
net = CHNet().to(DEVICE)
optimizer = RMSprop(net.parameters(), lr=LR)

In [72]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/10]. Step [1/241]. Loss: 0.078. Test acc: 0.088.
Epoch [1/10]. Step [151/241]. Loss: 0.015. Test acc: 0.008.
Epoch [1/10]. Step [241/241]. Loss: 0.009. Test acc: 0.010.
Epoch [2/10]. Step [1/241]. Loss: 0.009. Test acc: 0.010.
Epoch [2/10]. Step [151/241]. Loss: 0.008. Test acc: 0.010.
Epoch [2/10]. Step [241/241]. Loss: 0.007. Test acc: 0.007.
Epoch [3/10]. Step [1/241]. Loss: 0.011. Test acc: 0.006.
Epoch [3/10]. Step [151/241]. Loss: 0.007. Test acc: 0.006.
Epoch [3/10]. Step [241/241]. Loss: 0.007. Test acc: 0.006.
Epoch [4/10]. Step [1/241]. Loss: 0.006. Test acc: 0.006.
Epoch [4/10]. Step [151/241]. Loss: 0.007. Test acc: 0.008.
Epoch [4/10]. Step [241/241]. Loss: 0.007. Test acc: 0.007.
Epoch [5/10]. Step [1/241]. Loss: 0.007. Test acc: 0.007.
Epoch [5/10]. Step [151/241]. Loss: 0.007. Test acc: 0.006.
Epoch [5/10]. Step [241/241]. Loss: 0.007. Test acc: 0.011.
Epoch [6/10]. Step [1/241]. Loss: 0.006. Test acc: 0.011.
Epoch [6/10]. Step [151/241]. Loss: 0.006. Test acc:

### SGD

In [73]:
net = CHNet().to(DEVICE)
optimizer = SGD(net.parameters(), lr=LR)

In [74]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/10]. Step [1/241]. Loss: 0.050. Test acc: 0.088.
Epoch [1/10]. Step [151/241]. Loss: 0.025. Test acc: 0.011.
Epoch [1/10]. Step [241/241]. Loss: 0.011. Test acc: 0.011.
Epoch [2/10]. Step [1/241]. Loss: 0.010. Test acc: 0.010.
Epoch [2/10]. Step [151/241]. Loss: 0.010. Test acc: 0.010.
Epoch [2/10]. Step [241/241]. Loss: 0.010. Test acc: 0.009.
Epoch [3/10]. Step [1/241]. Loss: 0.009. Test acc: 0.008.
Epoch [3/10]. Step [151/241]. Loss: 0.009. Test acc: 0.008.
Epoch [3/10]. Step [241/241]. Loss: 0.008. Test acc: 0.009.
Epoch [4/10]. Step [1/241]. Loss: 0.010. Test acc: 0.009.
Epoch [4/10]. Step [151/241]. Loss: 0.008. Test acc: 0.006.
Epoch [4/10]. Step [241/241]. Loss: 0.008. Test acc: 0.008.
Epoch [5/10]. Step [1/241]. Loss: 0.006. Test acc: 0.007.
Epoch [5/10]. Step [151/241]. Loss: 0.007. Test acc: 0.007.
Epoch [5/10]. Step [241/241]. Loss: 0.007. Test acc: 0.007.
Epoch [6/10]. Step [1/241]. Loss: 0.008. Test acc: 0.007.
Epoch [6/10]. Step [151/241]. Loss: 0.007. Test acc:

### SGD + Momentum

In [75]:
net = CHNet().to(DEVICE)
optimizer = SGD(net.parameters(), lr=LR, momentum=0.8)

In [76]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/10]. Step [1/241]. Loss: 0.064. Test acc: 0.087.
Epoch [1/10]. Step [151/241]. Loss: 0.015. Test acc: 0.008.
Epoch [1/10]. Step [241/241]. Loss: 0.008. Test acc: 0.007.
Epoch [2/10]. Step [1/241]. Loss: 0.006. Test acc: 0.007.
Epoch [2/10]. Step [151/241]. Loss: 0.008. Test acc: 0.006.
Epoch [2/10]. Step [241/241]. Loss: 0.008. Test acc: 0.006.
Epoch [3/10]. Step [1/241]. Loss: 0.007. Test acc: 0.006.
Epoch [3/10]. Step [151/241]. Loss: 0.007. Test acc: 0.006.
Epoch [3/10]. Step [241/241]. Loss: 0.007. Test acc: 0.006.
Epoch [4/10]. Step [1/241]. Loss: 0.006. Test acc: 0.006.
Epoch [4/10]. Step [151/241]. Loss: 0.007. Test acc: 0.007.
Epoch [4/10]. Step [241/241]. Loss: 0.007. Test acc: 0.006.
Epoch [5/10]. Step [1/241]. Loss: 0.009. Test acc: 0.006.
Epoch [5/10]. Step [151/241]. Loss: 0.007. Test acc: 0.006.
Epoch [5/10]. Step [241/241]. Loss: 0.007. Test acc: 0.007.
Epoch [6/10]. Step [1/241]. Loss: 0.008. Test acc: 0.007.
Epoch [6/10]. Step [151/241]. Loss: 0.007. Test acc:

### Вывод

Поставим ограничение (выбирались рандомно): 
* Количество эпох - EPOCHES=10
* Размер батча в DataLoader - batch_size=64
* Скорость обучения - lr=0.01

*Вариация сетки* **первая**:

*Linear* > *BatchNorm1d* > *ReLU* > *Dropout*

*Сводка полученных данных*:
* Adam: 
  * скорость работы 27.8 s
  * наилучшая точность:
    * тренеровочный вариант 0.004
    * тестовый вариант 0.005
* RMSprop:
  * скорость работы 25.5 s
  * наилучшая точность:
    * тренеровочный вариант 0.004
    * тестовый вариант 0.006
* SGD:
  * скорость работы 24.7 s
  * наилучшая точность:
    * тренеровочный вариант 0.005
    * тестовый вариант 0.006
* SGD + Momentum:
  * скорость работы 26.3 s
  * наилучшая точность:
    * тренеровочный вариант 0.005
    * тестовый вариант 0.005

**Результат**: Наибыстрый вариант функции сходимости оказался: SGD, по скорости отстала функция Adam. Наилучшие точности (оценивается по тестовым данным) стали методы: SGD + Momentum и Adam. Получается, что более точная модель требует больше времени.

***Комментарий***: Попробовать изменить вариацию сетки.

*Вариация сетки* **вторая**:

*Linear* > *ReLU* > *BatchNorm1d* > *Dropout*

*Сводка полученных данных*:
* Adam: 
  * скорость работы 26.3 s
  * наилучшая точность:
    * тренеровочный вариант 0.004
    * тестовый вариант 0.005
* RMSprop:
  * скорость работы 24.8 s
  * наилучшая точность:
    * тренеровочный вариант 0.003
    * тестовый вариант 0.003
* SGD:
  * скорость работы 24.2 s
  * наилучшая точность:
    * тренеровочный вариант 0.004
    * тестовый вариант 0.004
* SGD + Momentum:
  * скорость работы 24.7 s
  * наилучшая точность:
    * тренеровочный вариант 0.004
    * тестовый вариант 0.006

**Результат**: Наибыстрый вариант функции сходимости оказался: SGD, по скорости отстала функция Adam. Наилучшая точность (оценивается по тестовым данным) стали методы: RMSProp. По сравнению с первой вариацией нейронной сети, в данной получилось ускорить работу по всем функциям сходимости и в некоторых получилось улучшить точность.

***Комментарий***: Если брать только по функции RMSProp, то выглядит оптимально... Стоит ли менять? Можно попробовать поменять вариацию.

*Вариация сетки* **третья**:

*Linear* > *Dropout* > *BatchNorm1d* > *ReLU*

*Сводка полученных данных*:
* Adam: 
  * скорость работы 28.1 s
  * наилучшая точность:
    * тренеровочный вариант 0.004
    * тестовый вариант 0.006
* RMSprop:
  * скорость работы 25.5 s
  * наилучшая точность:
    * тренеровочный вариант 0.003
    * тестовый вариант 0.003
* SGD:
  * скорость работы 25.7 s
  * наилучшая точность:
    * тренеровочный вариант 0.003
    * тестовый вариант 0.004
* SGD + Momentum:
  * скорость работы 25.2 s
  * наилучшая точность:
    * тренеровочный вариант 0.004
    * тестовый вариант 0.005

**Результат**: Наибыстрый вариант функции сходимости оказался: SGD+Momentum, по скорости отстала функция Adam. Наилучшая точность (оценивается по тестовым данным) стали методы: RMSProp. По сравнению с предыдущими эта сетка самая плохая на данный момент, т.к. время увеличилось, точнасть не сильно изменилась.

***Комментарий***: Хм... Нет... Просто нет :D Такую сетку использовать не вижу смысла, только для тестирования если только :D




